In [ ]:

import pandas as pd

# Load datasets
airlines = pd.read_csv('Data/airlines.csv', delimiter=';', quotechar='"', na_values='NA')
airports = pd.read_csv('Data/airports.csv', delimiter=';', quotechar='"', na_values='NA')
flights = pd.read_csv('Data/flights.csv', delimiter=';', quotechar='"', na_values='NA')
planes = pd.read_csv('Data/planes.csv', delimiter=';', quotechar='"', na_values='NA')
weather = pd.read_csv('Data/weather.csv', delimiter=';', quotechar='"', na_values='NA')

# Display basic information about the datasets
print("Airlines shape:", airlines.shape)
print("Airports shape:", airports.shape)
print("Flights shape:", flights.shape)
print("Planes shape:", planes.shape)
print("Weather shape:", weather.shape)


### Data Cleaning

In [ ]:
def clean_airports(df):
    df = df.dropna()
    df['faa'] = df['faa'].astype(str)
    df['name'] = df['name'].astype(str)
    df['lat'] = df['lat'].astype(str).str.replace(',', '.').astype(float)
    df['lon'] = df['lon'].astype(str).str.replace(',', '.').astype(float)
    df['alt'] = df['alt'].astype(str).str.replace(',', '.').astype(float)
    df['tz'] = df['tz'].astype(int)
    df['dst'] = df['dst'].astype(str)
    df['tzone'] = df['tzone'].astype(str)
    return df


def clean_flights(df):
    df = df.dropna()
    df['year'] = df['year'].astype(int)
    df['month'] = df['month'].astype(int)
    df['day'] = df['day'].astype(int)
    df['dep_time'] = df['dep_time'].astype(int)
    df['sched_dep_time'] = df['sched_dep_time'].astype(int)
    df['dep_delay'] = df['dep_delay'].astype(int)
    df['arr_time'] = df['arr_time'].astype(int)
    df['sched_arr_time'] = df['sched_arr_time'].astype(int)
    df['arr_delay'] = df['arr_delay'].astype(int)
    df['carrier'] = df['carrier'].astype(str)
    df['flight'] = df['flight'].astype(int)
    df['tailnum'] = df['tailnum'].astype(str)
    df['origin'] = df['origin'].astype(str)
    df['dest'] = df['dest'].astype(str)
    df['air_time'] = df['air_time'].astype(str).str.replace(',', '.').astype(float)
    df['distance'] = df['distance'].astype(str).str.replace(',', '.').astype(float)
    df['hour'] = df['hour'].astype(int)
    df['minute'] = df['minute'].astype(int)
    df['time_hour'] = pd.to_datetime(df['time_hour'])  # Convert to datetime
    return df


def clean_planes(df):
    df = df.dropna()
    df['tailnum'] = df['tailnum'].astype(str)
    df['year'] = df['year'].astype(int)
    df['type'] = df['type'].astype(str)
    df['manufacturer'] = df['manufacturer'].astype(str)
    df['model'] = df['model'].astype(str)
    df['engines'] = df['engines'].astype(int)
    df['seats'] = df['seats'].astype(int)
    df['speed'] = df['speed'].astype(str).str.replace(',', '.').astype(float)
    df['engine'] = df['engine'].astype(str)
    return df


def clean_weather(df):
    # These columns are almost completely filled with NA.
    # The number of non-NA fields is too small to conclude anything, so we are dropping them
    df = df.drop(columns=['temp', 'dewp', 'humid', 'precip', 'pressure'])
    df = df.dropna()
    df['year'] = df['year'].astype(int)
    df['month'] = df['month'].astype(int)
    df['day'] = df['day'].astype(int)
    df['hour'] = df['hour'].astype(int)
    df['wind_dir'] = df['wind_dir'].astype(int)
    df['wind_speed'] = df['wind_speed'].astype(str).str.replace(',', '.').astype(float)
    df['wind_gust'] = df['wind_gust'].astype(str).str.replace(',', '.').astype(float)
    df['visib'] = df['visib'].astype(str).str.replace(',', '.').astype(float)
    df['time_hour'] = pd.to_datetime(df['time_hour'], errors='coerce')
    return df

In [ ]:
# Apply cleaning functions
airports_cleaned = clean_airports(airports)
flights_cleaned = clean_flights(flights)
planes_cleaned = clean_planes(planes)
weather_cleaned = clean_weather(weather)

tables = {"Airports": (airports, airports_cleaned), "Flights": (flights, flights_cleaned),
          "Planes": (planes, planes_cleaned), "Weather": (weather, weather_cleaned)}
for name, table in tables.items():
    print(f"{name} before cleaning:")
    print(table[0].info(), end="\n")
    print(f"{name} after cleaning:")
    print(table[1].info(), end="\n")

### ExploratoryDataAnalysis(EDA)

In [ ]:
%pip install matplotlib seaborn

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


# Summarize Datasets
def dataset_summary(df, name):
    print(f"Summary for {name}:")
    print(df.describe(include='all'))
    print(f"\nMissing values in {name}:\n{df.isnull().sum()}\n")
    print("-" * 50)


# Airports Dataset EDA
dataset_summary(airports_cleaned, "Airports")
plt.figure(figsize=(10, 6))
sns.histplot(airports_cleaned['alt'], kde=True, bins=30)
plt.title("Distribution of Airport Altitudes")
plt.xlabel("Altitude (ft)")
plt.ylabel("Frequency")
plt.show()

# Flights Dataset EDA
dataset_summary(flights_cleaned, "Flights")
plt.figure(figsize=(10, 6))
sns.histplot(flights_cleaned['arr_delay'], kde=True, bins=50, color='skyblue')
plt.title("Distribution of Arrival Delays")
plt.xlabel("Arrival Delay (minutes)")
plt.ylabel("Frequency")
plt.xlim(flights_cleaned['arr_delay'].min(), flights_cleaned['arr_delay'].max())
plt.show()

# Explore delay trends by month
plt.figure(figsize=(10, 6))
sns.boxplot(data=flights_cleaned, x='month', y='arr_delay', palette='coolwarm')
plt.title("Arrival Delay by Month")
plt.xlabel("Month")
plt.ylabel("Arrival Delay (minutes)")
plt.show()

# Planes Dataset EDA
dataset_summary(planes_cleaned, "Planes")
plt.figure(figsize=(10, 6))
sns.countplot(data=planes_cleaned, x='engines', palette='viridis')
plt.title("Number of Engines in Planes")
plt.xlabel("Engines")
plt.ylabel("Count")
plt.show()

# Weather Dataset EDA
dataset_summary(weather_cleaned, "Weather")
plt.figure(figsize=(10, 6))
sns.histplot(weather_cleaned['wind_speed'], kde=True, bins=50, color='orange')
plt.title("Distribution of Temperature")
plt.xlabel("Wind speed (mph)")
plt.ylabel("Frequency")
plt.show()

# Correlation Analysis for Flights
corr = flights_cleaned[['arr_delay', 'dep_delay', 'air_time', 'distance']].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix for Flights")
plt.show()

In [ ]:
df = flights_cleaned.copy()


# Features and target
X = weather_cleaned[['wind_dir', 'wind_speed', 'wind_gust', 'visib']]
y = weather_cleaned['month']

In [ ]:
%pip install scikit-learn

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing pipeline: Scale numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Logistic Regression
logistic_model = LogisticRegression(multi_class='multinomial', max_iter=1000, random_state=42)
logistic_model.fit(X_train_scaled, y_train)
y_pred_logistic = logistic_model.predict(X_test_scaled)

print("Logistic Regression Results:")
print(classification_report(y_test, y_pred_logistic))
print("Accuracy:", accuracy_score(y_test, y_pred_logistic))

# Random Forest Classifier
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_scaled, y_train)
y_pred_rf = rf_model.predict(X_test_scaled)

print("\nRandom Forest Results:")
print(classification_report(y_test, y_pred_rf))
print("Accuracy:", accuracy_score(y_test, y_pred_rf))